#LLM Finetuning Notebook

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
from google.colab import userdata

import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-14B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = userdata.get('HF_token'), # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/196k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.4.7 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
import pandas as pd
df = pd.read_csv('/content/analysed_jobs_with_analysis.csv')

In [ ]:
alpaca_prompt = """You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters. Your task is to extract key details from a given job description and format them into a JSON object with the following structure:

{{
  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",
  "key_terms": [
    {{
      "term": "Technical Term or jargon from the job description",
      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."
    }}
  ],
  "skill_priorities": {{
    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],
    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]
  }},
  "proposed_screening_questions_with_answers": [
    {{
      "question": "A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.",
      "example_answer": "An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role."
    }}
  ],
  "red_flags": [
    "Indicators of potential mismatches for the role, offering actionable insights for recruiters based on the job offer. Examples include 'Avoid candidates without cloud experience'."
  ],
  "confidence_score": "A numerical score between 0 and 100 indicating the model's confidence in the accuracy of its analysis."
}}

Guidelines:
- Ensure the role summary is written in simple, non-technical language, focusing on primary responsibilities rather than qualifications.
- Extract key technical terms and provide clear, non-technical explanations.
- Identify and categorize skills into 'must-have' and 'nice-to-have' based on explicit mentions in the job description.
- Generate screening questions that assess relevant competencies and provide an example of a strong answer.
- Highlight red flags that indicate potential mismatches, ensuring they are actionable.
- Assign a confidence score (0-100) reflecting the reliability of the extracted insights.

Output only the JSON object without additional text.

### job description:
{}

### Response:
{}

"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Details"]
    outputs      = examples["parsed_description"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [ ]:
from datasets import Dataset
# Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/370 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
print(type(tokenizer))  # Should show <class 'transformers...Tokenizer'>

<class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Set to `True` for masked language modeling
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator=data_collator,  # Add this line
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 4321,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/370 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
12.469 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 370 | Num Epochs = 1 | Total steps = 46
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 68,812,800/14,000,000,000 (0.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.565100
2,1.598800
3,1.523700
4,1.589000
5,1.429500
6,1.447500
7,1.416400
8,1.351600
9,1.370200
10,1.145600


Step,Training Loss
1,1.565100
2,1.598800
3,1.523700
4,1.589000
5,1.429500
6,1.447500
7,1.416400
8,1.351600
9,1.370200
10,1.145600


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

3444.5343 seconds used for training.
57.41 minutes used for training.
Peak reserved memory = 14.57 GB.
Peak reserved memory for training = 2.101 GB.
Peak reserved memory % of max memory = 98.84 %.
Peak reserved memory for training % of max memory = 14.253 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        """
        About the job
        Infor Decision Analytics and Science (IDeAS) is a division of Infor that provides AI/ML solutions for various industries. We use advanced predictive and prescriptive analytics to create value-driven solutions. Our goal is to enable fast and effective decision automation for all business functions.

        The Data Science Engineers in IDEAS are the brain behind every data-driven solution. This role relies on a combination of analytics and technical expertise to turn data vision into reality. As a Senior Data Science Engineer, you are responsible for designing, building, and operationalizing large-scale predictive and prescriptive analytics to solve business problems that are important to Infor customers.

        A Day in The Life Typically Includes:
        * Architect, implement, and oversee the end-to-end lifecycle of advanced machine learning solutions, including ML model orchestration, scalable data pipelines, and performance optimization.
        * Develop and enhance ETL/ELT pipelines from diverse customer data sources to ensure alignment with complex ML solution requirements.
        * Drive strategic discussions with business stakeholders to identify critical business challenges and opportunities addressable through AI/ML.
        * Articulate results, insights, and actionable recommendations to executives and cross-functional teams, ensuring alignment with business goals.
        * Lead exploratory data analysis on large, complex, and diverse datasets to evaluate data quality, derive insights, and inform model development.
        * Innovate by designing and delivering cutting-edge AI/ML solutions tailored to specific industry needs, leveraging the latest technologies and frameworks.
        * Mentor and provide technical guidance to junior team members, fostering skill development and ensuring high-quality deliverables across projects.

        Basic Qualifications:
        * Engineering degree (or foreign equivalent) in Mathematical Modeling, Operations Research, Computer Science, Telecommunications, Electrical Engineering, Industrial Engineering, or a related quantitative discipline from an accredited university (or foreign equivalent).
        * Extensive hands-on experience in architecting and deploying end-to-end machine learning and/or optimization pipelines, including ETL processes, data pre-processing, exploratory data analysis, model development, deployment, and production inference.
        * Expert programming skills in Python and strong data-querying expertise in SQL, PySpark, or equivalent frameworks.
        * Proven ability to effectively communicate complex technical concepts and solutions through presentations, technical documentation, blog posts, GitHub projects, and other mediums.
        * Good verbal and written communication skills, with the ability to distill and convey intricate technical topics to diverse audiences, including scientists, engineers, and business leaders.
        * Advanced data modeling and analytical problem-solving skills with a focus on delivering scalable and impactful solutions.
        * Strong collaborative mindset with demonstrated interpersonal skills to lead and contribute effectively to cross-functional teams addressing complex business challenges.
        * Proficiency in analytics and visualization tools such as Tableau, Power BI, or similar platforms, with a track record of creating actionable insights.

        Preferred Qualifications:
        * Advanced degree (Master's or PhD) in Mathematical Modeling, Operations Research, Computer Science, Telecommunications, Electrical Engineering, Industrial Engineering, or a related quantitative discipline from an accredited university (or foreign equivalent).
        * Advanced expertise in machine learning algorithms and hands-on experience with industry-standard frameworks such as TensorFlow, PyTorch, Keras, or equivalent.
        * Demonstrated experience in designing, implementing, and optimizing time series forecasting models and personalized recommendation systems at scale.
        * Proven track record in developing and deploying AI/ML solutions for real-world business problems, including but not limited to demand forecasting, anomaly detection, pricing optimization, and recommendation engines.

        About Infor

        Infor is a global leader in business cloud software products for companies in industry specific markets. Infor builds complete industry suites in the cloud and efficiently deploys technology that puts the user experience first, leverages data science, and integrates easily into existing systems. Over 60,000 organizations worldwide rely on Infor to help overcome market disruptions and achieve business-wide digital transformation.
        For more information visit www.infor.com

        Our Values

        At Infor, we strive for an environment that is founded on a business philosophy called [1] Principle Based Management™ (PBM™) and eight Guiding Principles: integrity, stewardship & compliance, transformation, principled entrepreneurship, knowledge, humility, respect, self-actualization. Increasing diversity is important to reflect our markets, customers, partners, and communities we serve in now and in the future.

        We have a relentless commitment to a culture based on PBM. Informed by the principles that allow a free and open society to flourish, PBM™ prepares individuals to innovate, improve, and transform while fostering a healthy, growing organization that creates long-term value for its clients and supporters and fulfillment for its employees.

        Infor is an Equal Opportunity Employer. We are committed to creating a diverse and inclusive work environment. Infor does not discriminate against candidates or employees because of their sex, race, gender identity, disability, age, sexual orientation, religion, national origin, veteran status, or any other protected status under the law. If you require accommodation or assistance at any time during the application or selection processes, please submit a request by following the directions located in the FAQ section at the bottom of the infor.com/about/careers webpage.

        At Infor we value your privacy that’s why we created a policy that you can read [2] here.


        References

        Visible links
        1. https://www.kochind.com/about/business-philosophy
        2. https://www.infor.com/about/privacy
        """, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1200, use_cache = True)
tokenizer.batch_decode(outputs)

['You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters. Your task is to extract key details from a given job description and format them into a JSON object with the following structure:\n\n{\n  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",\n  "key_terms": [\n    {\n      "term": "Technical Term or jargon from the job description",\n      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."\n    }\n  ],\n  "skill_priorities": {\n    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],\n    "nice_to_have": ["List of preferred skills that are beneficial but no

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
model.push_to_hub("HichamD1/Qwen2.5-14B-lora", token = userdata.get('HF_token')) # Online saving
tokenizer.push_to_hub("HichamD1/Qwen2.5-14B-lora", token = userdata.get('HF_token')) # Online saving

README.md:   0%|          | 0.00/593 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/275M [00:00<?, ?B/s]

Saved model to https://huggingface.co/HichamD1/Qwen2.5-14B-lora


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
import json
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "AhmedBou/Qwen2.5-14B-lora",
        max_seq_length = 2048,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        """
        About the job
        Infor Decision Analytics and Science (IDeAS) is a division of Infor that provides AI/ML solutions for various industries. We use advanced predictive and prescriptive analytics to create value-driven solutions. Our goal is to enable fast and effective decision automation for all business functions.

        The Data Science Engineers in IDEAS are the brain behind every data-driven solution. This role relies on a combination of analytics and technical expertise to turn data vision into reality. As a Senior Data Science Engineer, you are responsible for designing, building, and operationalizing large-scale predictive and prescriptive analytics to solve business problems that are important to Infor customers.

        A Day in The Life Typically Includes:
        * Architect, implement, and oversee the end-to-end lifecycle of advanced machine learning solutions, including ML model orchestration, scalable data pipelines, and performance optimization.
        * Develop and enhance ETL/ELT pipelines from diverse customer data sources to ensure alignment with complex ML solution requirements.
        * Drive strategic discussions with business stakeholders to identify critical business challenges and opportunities addressable through AI/ML.
        * Articulate results, insights, and actionable recommendations to executives and cross-functional teams, ensuring alignment with business goals.
        * Lead exploratory data analysis on large, complex, and diverse datasets to evaluate data quality, derive insights, and inform model development.
        * Innovate by designing and delivering cutting-edge AI/ML solutions tailored to specific industry needs, leveraging the latest technologies and frameworks.
        * Mentor and provide technical guidance to junior team members, fostering skill development and ensuring high-quality deliverables across projects.

        Basic Qualifications:
        * Engineering degree (or foreign equivalent) in Mathematical Modeling, Operations Research, Computer Science, Telecommunications, Electrical Engineering, Industrial Engineering, or a related quantitative discipline from an accredited university (or foreign equivalent).
        * Extensive hands-on experience in architecting and deploying end-to-end machine learning and/or optimization pipelines, including ETL processes, data pre-processing, exploratory data analysis, model development, deployment, and production inference.
        * Expert programming skills in Python and strong data-querying expertise in SQL, PySpark, or equivalent frameworks.
        * Proven ability to effectively communicate complex technical concepts and solutions through presentations, technical documentation, blog posts, GitHub projects, and other mediums.
        * Good verbal and written communication skills, with the ability to distill and convey intricate technical topics to diverse audiences, including scientists, engineers, and business leaders.
        * Advanced data modeling and analytical problem-solving skills with a focus on delivering scalable and impactful solutions.
        * Strong collaborative mindset with demonstrated interpersonal skills to lead and contribute effectively to cross-functional teams addressing complex business challenges.
        * Proficiency in analytics and visualization tools such as Tableau, Power BI, or similar platforms, with a track record of creating actionable insights.

        Preferred Qualifications:
        * Advanced degree (Master's or PhD) in Mathematical Modeling, Operations Research, Computer Science, Telecommunications, Electrical Engineering, Industrial Engineering, or a related quantitative discipline from an accredited university (or foreign equivalent).
        * Advanced expertise in machine learning algorithms and hands-on experience with industry-standard frameworks such as TensorFlow, PyTorch, Keras, or equivalent.
        * Demonstrated experience in designing, implementing, and optimizing time series forecasting models and personalized recommendation systems at scale.
        * Proven track record in developing and deploying AI/ML solutions for real-world business problems, including but not limited to demand forecasting, anomaly detection, pricing optimization, and recommendation engines.

        About Infor

        Infor is a global leader in business cloud software products for companies in industry specific markets. Infor builds complete industry suites in the cloud and efficiently deploys technology that puts the user experience first, leverages data science, and integrates easily into existing systems. Over 60,000 organizations worldwide rely on Infor to help overcome market disruptions and achieve business-wide digital transformation.
        For more information visit www.infor.com

        Our Values

        At Infor, we strive for an environment that is founded on a business philosophy called [1] Principle Based Management™ (PBM™) and eight Guiding Principles: integrity, stewardship & compliance, transformation, principled entrepreneurship, knowledge, humility, respect, self-actualization. Increasing diversity is important to reflect our markets, customers, partners, and communities we serve in now and in the future.

        We have a relentless commitment to a culture based on PBM. Informed by the principles that allow a free and open society to flourish, PBM™ prepares individuals to innovate, improve, and transform while fostering a healthy, growing organization that creates long-term value for its clients and supporters and fulfillment for its employees.

        Infor is an Equal Opportunity Employer. We are committed to creating a diverse and inclusive work environment. Infor does not discriminate against candidates or employees because of their sex, race, gender identity, disability, age, sexual orientation, religion, national origin, veteran status, or any other protected status under the law. If you require accommodation or assistance at any time during the application or selection processes, please submit a request by following the directions located in the FAQ section at the bottom of the infor.com/about/careers webpage.

        At Infor we value your privacy that’s why we created a policy that you can read [2] here.


        References

        Visible links
        1. https://www.kochind.com/about/business-philosophy
        2. https://www.infor.com/about/privacy
        """, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


# Generate the output
outputs = model.generate(**inputs, max_new_tokens=1200, use_cache=True)

# Decode the outputs
decoded_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

# Split by "### Response:" and get the last part
split_text = decoded_text.rsplit("### Response:", 1)  # Splitting from the right
if len(split_text) > 1:
    response_text = split_text[1].strip()  # Extract and clean the response
else:
    response_text = ""


parsed_json = json.loads(response_text)
print(json.dumps(parsed_json, indent=2))


{
  "role_summary": "A Senior Data Science Engineer designs, builds, and operationalizes large-scale predictive and prescriptive analytics to solve business problems for Infor customers.",
  "key_terms": [
    {
      "term": "Machine Learning",
      "explanation": "A type of artificial intelligence that enables systems to learn from data and improve their performance over time."
    },
    {
      "term": "ETL/ELT Pipelines",
      "explanation": "Data integration processes that extract, transform, and load data from various sources into a centralized repository."
    },
    {
      "term": "Predictive and Prescriptive Analytics",
      "explanation": "Techniques that use data and statistical models to forecast future outcomes and provide actionable recommendations."
    },
    {
      "term": "Time Series Forecasting",
      "explanation": "A method of predicting future values based on historical data, often used in demand forecasting and anomaly detection."
    },
    {
      "term

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("HichamD1/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("HichamD1/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("HichamD1/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if True:
    model.push_to_hub_gguf(
        "HichamD1/Qwen2.5-7B-gguf", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = userdata.get('HF_token'), # Get a token at https://huggingface.co/settings/tokens
    )

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.86 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 10%|█         | 5/48 [01:06<09:30, 13.27s/it]


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 2.85 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  4%|▍         | 2/48 [00:26<10:07, 13.21s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 272.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 124.12 MiB is free. Process 3713 has 14.61 GiB memory in use. Of the allocated memory 14.10 GiB is allocated by PyTorch, and 362.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)